<a href="https://colab.research.google.com/github/ailunguo/Test/blob/main/Tensorflow_Test/%E8%BF%81%E7%A7%BB%E5%AD%A6%E4%B9%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

常见的迁移学习的工作流程

1，从先前的训练模型中获取图层

2，冻结它们，以避免在以后的训练中破坏它们包含的任何信息

3，在冻结层之上添加一些新的可训练层，它们将学习将旧特征转化为对新数据集的预测

4，在数据集上训练新图层

最后一个步骤就是微调，其中包括解冻上面获得的整个模型(或部分模型),并以非常低的学习率在新数据上重新训练它。通过逐步使预训练的特征适应新数据，这可能会实现有意义的改进。

In [1]:
import numpy as np
import tensorflow as tf
import keras

# 冻结层: 理解trainable属性

图层和模型具有三个权重属性：

weights是该层所有权重变量的列表。

trainable_weights是要更新（通过梯度下降）以最小化训练期间损失的列表。

non_trainable_weights是那些不应该接受培训的人员的列表。通常，它们由模型在前向传播期间更新。

In [3]:
# 该Dense有两个可训练的权重(内核和偏差)
layer = keras.layers.Dense(3)
layer.build((None, 4))

print('weights:', len(layer.weights))
print('trainable_weights:', len(layer.trainable_weights))
print('non_trainable_weights:', len(layer.non_trainable_weights))

weights: 2
trainable_weights: 2
non_trainable_weights: 0


In [4]:
# 该BatchNormalization层有 2 个可训练权重和 2 个不可训练权重
layer = keras.layers.BatchNormalization()
layer.build((None, 4))  # Create the weights

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

weights: 4
trainable_weights: 2
non_trainable_weights: 2


In [5]:
# 设置trainable为False
layer = keras.layers.Dense(3)
layer.build((None, 4))  # Create the weights
layer.trainable = False  # Freeze the layer

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

weights: 2
trainable_weights: 0
non_trainable_weights: 2


当可训练权重变为不可训练权重时，其值在训练期间不再更新。

In [6]:
layer1 = keras.layers.Dense(3, activation='relu')
layer2 = keras.layers.Dense(3, activation='sigmoid')
model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])

# 将layer1层冻结
layer1.trainable = False

# 将layer1的权重复制一下
initial_layer1_weights_values = layer1.get_weights()

# 训练模型
model.compile(optimizer='adam', loss='mse')
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

final_layer1_weights_values = layer1.get_weights()
np.testing.assert_allclose(
    initial_layer1_weights_values[0], final_layer1_weights_values[0]
)
np.testing.assert_allclose(
    initial_layer1_weights_values[1], final_layer1_weights_values[1]
)

1/1 [==============================] - 1s 503ms/step - loss: 0.1272


# trainable属性的递归设置

如果您trainable = False在模型或任何具有子层的层上进行设置，则所有子层也将变得不可训练。

In [7]:
inner_model = keras.Sequential(
    [
        keras.Input(shape=(3,)),
        keras.layers.Dense(3, activation='relu'),
        keras.layers.Dense(3, activation='relu'),
    ]
)

model = keras.Sequential(
    [
        keras.Input(shape=(3,)),
        inner_model,
        keras.layers.Dense(3, activation='sigmoid'),
    ]
)

model.trainable = False  # 将整个模型的trainable设置为False

assert inner_model.trainable == False # 所有模型都被冻结
assert inner_model.layers[0].trainable == False # 'trainable'是具有递归性质的

# Keras迁移学习流程

使用预先训练的权重实例化基本模型

In [10]:
base_model = keras.applications.Xception(
    weights = 'imagenet',
    input_shape = (150, 150, 3),
    include_top = False
)

# 然后冻结该基本模型
base_model.trainable = False

83683744/83683744 [==============================] - 4s 0us/step


在此基础上创建一个新的模型

In [12]:
# 在基本模型之上创建一个新的模型
inputs = keras.Input(shape=(150, 150, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x) # 一个新的全局平均池化层
outputs = keras.layers.Dense(1)(x)  # 最后输出层输出一个
model = keras.Model(inputs, outputs) # 这是个新的模型

使用新模型训练数据

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
       loss=keras.losses.BinaryCrossentropy(from_logits=True),
       metrics=[keras.metrics.BinaryAccuracy()])
# 选择是否设置from_logits参数通常取决于模型的最后一层以及任务的特性。
# 如果模型的最后一层包含sigmoid激活函数，并输出概率值（范围在0到1之间），
# 则通常将from_logits设置为False。如果最后一层输出未经处理的 logits，
# 通常将from_logits设置为 True，以确保在计算损失时应用适当的转换。

model.fit(new_dataset, epochs=20, callbacks=..., validation_data=...)

# 微调

1，一旦您的模型收敛于新数据，您可以尝试解冻全部或部分基础模型，并以非常低的学习率端到端地重新训练整个模型。

2，这是可选的最后一步，可能会给您带来增量改进。***它还可能导致快速过度拟合***

3，**只有在具有冻结层的模型经过训练以收敛之后才执行此步骤**

4，在此阶段使用***非常低的学习率***也很重要，因为您要在通常*非常小的数据集上训练比第一轮训练大得多的模型*。

In [ ]:
# 解冻基础模型
# 还要再重新编译解冻后的模型
base_model.trainable = True

model.compile(optimizer=keras.optimizers.Adam(1e-5),
       loss=keras.losses.BinaryCrossentropy(from_logits=True),
       metrics=[keras.metrics.BinaryAccuracy()])
model.fit(new_dataset, epochs=10, callbacks=..., validation_data=...)

如果您更改任何trainable值，请确保compile()再次调用您的模型以使更改生效。

-------------------------------------------------------------------------------

# 通过自定义训练循环进行迁移学习和微调

In [ ]:
# 创建基础模型
base_model = keras.applications.Xception(
    weights = 'imagenet',
    input_shape = (150, 150, 3),
    include_top = False
)
base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
x = base_model(inputs, training=False) # training=False告诉call方法，无论是训练还是推理都不更新权重
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()

for inputs, targets in new_dataset:
  with tf.GradientTape() as tape:
    pred = model(inputs)
    loss_val = loss_fn(targets, pred)

  gradients = tape.gradient(loss_val, model.trainable_weights)
  optimizer.apply_gradients(zip(gradiens, model.trainable_weights))